# **Model Training**

## Objectives

* To train, evaluate, and compare predictive models that estimate house sale prices using the engineered datasets created in 04_feature_engineering.ipynb.
* I will build two regression models — Linear Regression (using scaled features) and Random Forest Regressor (using unscaled features) — and assess their performance on the Ames housing dataset.
* The final step will be to select the best-performing model and use it to generate sale price predictions for the client's four inherited houses.



## Inputs

* This notebook uses the processed datasets stored in `data/processed/`, all created in the feature engineering notebook:

  * `train_engineered.csv` (unscaled training set)
  * `test_engineered.csv` (unscaled test set)
  * `train_engineered_scaled.csv` (scaled training set for Linear Regression)
  * `test_engineered_scaled.csv` (scaled test set for Linear Regression)
  * `inherited_engineered.csv` (unscaled engineered features for predictions)
  * `inherited_engineered_scaled.csv` (scaled version for predictions)


## Outputs

* Fitted Linear Regression and Random Forest models
* A performance comparison using R² and RMSE on the test data
* Identification of the preferred model for this project
* A final table containing the predicted sale prices for the four inherited houses based on the selected model

## Additional Comments

* This notebook focuses exclusively on **model training, evaluation, and prediction**, keeping the modelling process reproducible and separate from data preparation.
* The inherited houses are treated as unseen data and are only used for final prediction, not for training or validation.

---